In [77]:
import os
import torch
import matplotlib.pyplot as plt
import xml.dom.minidom
from skimage.io import imread
from skimage import img_as_ubyte
from pyrr import Matrix44, Vector4, Vector3
import pyrr

# Util function for loading meshes
from pytorch3d.io import load_objs_as_meshes

# Data structures and functions for rendering
from pytorch3d.structures import Meshes, Textures
from pytorch3d.renderer import (
    OpenGLPerspectiveCameras, 
    PointLights, 
    DirectionalLights, 
    Materials, 
    RasterizationSettings, 
    MeshRenderer, 
    MeshRasterizer,  
    TexturedSoftPhongShader,
    HardPhongShader
)
from pytorch3d.renderer.cameras import look_at_view_transform

from torch_openpose.body import Body
from torch_openpose import util
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import cm

from tqdm import tqdm
import torch.nn as nn
import imageio
import cv2
import copy

import wandb

# add path for demo utils functions 
import sys
import os
sys.path.append(os.path.abspath(''))
import pytorch3d

In [74]:
def transform_node_to_matrix(node):
    model_matrix = np.identity(4)
    for child in reversed(node.childNodes):
        if child.nodeName == "translate":
            x = float(child.getAttribute('x'))
            y = float(child.getAttribute('y'))
            z = float(child.getAttribute('z'))
            z *= -1
            translate_vec = Vector3([x, y, z])
            trans_matrix = np.transpose(pyrr.matrix44.create_from_translation(translate_vec))
            model_matrix = np.matmul(model_matrix, trans_matrix)
        if child.nodeName == "scale":
            scale = float(child.getAttribute('value'))
            scale_vec = Vector3([scale, scale, scale])
            scale_matrix = np.transpose(pyrr.matrix44.create_from_scale(scale_vec))
            model_matrix = np.matmul(model_matrix, scale_matrix)
            
    return model_matrix

def transform_node_to_R_T(node):
    eye = node.getAttribute('origin').split(',')
    eye = [float(i) for i in eye]
    at = node.getAttribute('target').split(',')
    at = [float(i) for i in at]
    up = node.getAttribute('up').split(',')
    up = [float(i) for i in up]
    
    R, T = look_at_view_transform(
        eye=[eye], 
        at=[at], 
        up=[up]
    )
    return R, T

def load_shape(shape):
    if shape is None:
        return None
    device = torch.device("cuda:0")
    char_filename_node = shape.getElementsByTagName('string')[0]
    char_filename = char_filename_node.getAttribute('value')
    obj_filename = os.path.join(DATA_ROOT, char_filename)
    mesh = load_objs_as_meshes([obj_filename], device=device)
    verticies, faces = mesh.get_mesh_verts_faces(0)
    texture = mesh.textures.clone() if mesh.textures is not None else None
    
    
    
    transform_node = shape.getElementsByTagName('transform')
    if len(transform_node) == 0:
        verticies[:, 2] *= -1
        return verticies, faces, texture
    # apply transform
    transform_node = transform_node[0]
    model_matrix =  transform_node_to_matrix(transform_node)
    model_matrix = torch.from_numpy(model_matrix).cuda().double() 
    
    # make coordiantes homegenos
    new_row = torch.ones(1, verticies.shape[0], device=device)
    vetrices_homo = torch.cat((verticies.t(), new_row)).double() 
    
    # transform
    vetrices_world = torch.chain_matmul(model_matrix, vetrices_homo).t()[:, :3]
    return vetrices_world.float(), faces, texture

def mitsuba_scene_to_torch_3d(master_scene):
    device = torch.device("cuda:0")
    master_doc = xml.dom.minidom.parse(master_scene)
    camera = master_doc.getElementsByTagName('sensor')[0]
    camera_transform = camera.getElementsByTagName('transform')[0]
    R, T = transform_node_to_R_T(camera_transform.getElementsByTagName('lookat')[0])
    # TODO figure out how to set camera params
    cameras = OpenGLPerspectiveCameras(
        znear=0.1,
        zfar=1000,
        fov=15,
        degrees=True,
        device=device, 
        R=R, 
        T=T
    )
    
    
    character = None
    tshirt = None
    ground = None
    
    shapes = master_doc.getElementsByTagName('shape')
    for i in range(len(shapes)):
        if shapes[i].getAttribute("id") == 'character':
            character = shapes[i]
        if shapes[i].getAttribute("id") == 'simulated':
            tshirt = shapes[i]
        if shapes[i].getAttribute("id") == 'place.000000':
            ground = shapes[i]
    
    tshirt_vetrices, tshirt_faces, tshirt_texture = load_shape(tshirt)
    character_vetrices, character_faces, character_texture = load_shape(character)
    ground_vetrices, ground_faces, ground_texture = load_shape(ground)
    
    texTshirt = torch.ones_like(tshirt_vetrices).cuda()
    tex2 = torch.ones_like(character_vetrices).cuda()
    tex3 = torch.ones_like(ground_vetrices).cuda()
    texTshirt[:, 1:] *= 0.0  # red
    # person
    tex2[:, 0] *= 0.88
    tex2[:, 1] *= 0.67
    tex2[:, 2] *= 0.41
    #tex3 [:, :2] *= 0.3  # blue
    tex = torch.cat([texTshirt, tex2, tex3])[None]  # (1, 204, 3)
    textures = Textures(verts_rgb=tex.cuda())
    
    verts = torch.cat([tshirt_vetrices,character_vetrices, ground_vetrices]).cuda()  #(204, 3)
    
    character_faces = character_faces + tshirt_vetrices.shape[0]  
    ground_faces = ground_faces  + character_vetrices.shape[0] + tshirt_vetrices.shape[0]  
    faces = torch.cat([tshirt_faces, character_faces, ground_faces]).cuda()  
    mesh = Meshes(verts=[verts], faces=[faces], textures=textures)
    
    optmization_input = {
        "texTshirt": texTshirt,
        "texOther": torch.cat([tex2, tex3]),
        "verticies": verts, 
        "faces": faces
    }
        
    return mesh, cameras, optmization_input

class Model(nn.Module):
    def __init__(self, texTshirt, texOther, verticies, faces, renderer, body_estimation):
        super().__init__()
        self.device = torch.device("cuda:0")
        self.renderer = renderer
        self.texOther = texOther
        self.verticies = verticies
        self.faces = faces
        self.body_estimation = body_estimation
        self.cur_mesh = None
        
        self.texTshirt = nn.Parameter(
            torch.zeros_like(texTshirt).to(self.device)
        )
        
        ## loss functions
        self.objective = torch.nn.MSELoss()
        self.zero_heatmap = torch.zeros((1, 19, 64, 64), device=self.device)
        self.zero_paf = torch.zeros((1, 38, 64, 64), device=self.device)

    def forward(self):
        tex = torch.cat([self.texTshirt, self.texOther])[None]  # (1, 204, 3)
        textures = Textures(verts_rgb=tex.cuda())
        
        self.cur_mesh = Meshes(verts=[self.verticies], faces=[self.faces], textures=textures)
        images = self.renderer(self.cur_mesh)
        
        #
        body_input = (images[..., :3] - 0.5).permute((0, 3, 1, 2)).float()
        paf, heat = self.body_estimation.model(body_input)
        
        ob_val_heat = self.objective(heat, self.zero_heatmap)
        ob_val_paf = self.objective(paf, self.zero_paf)
        
        # Calculate the silhouette loss
        loss = ob_val_heat + ob_val_paf
        return loss, images
    
def get_body_image_from_mesh(cur_mesh, body_estimation, renderer):
    images = renderer(cur_mesh)
        
    rendering_torch_input = (images[..., :3] - 0.5).permute((0, 3, 1, 2)).float()
    
    heatmap_avg, paf_avg = body_estimation.compute_heatmap_paf_avg(rendering_torch_input, ORIG_SHAPE)
    candidate, subset = body_estimation.get_pose(heatmap_avg, paf_avg, ORIG_SHAPE)
    rendering_torch_np =  images[0, ..., :3].detach().squeeze().cpu().numpy()
    canvas = copy.deepcopy(rendering_torch_np)
    canvas = util.draw_bodypose(canvas, candidate, subset)
    
    return canvas, candidate, subset, heatmap_avg, paf_avg

def wandb_init(name):
    run = wandb.init(
        project = "mts_tshirt_single", 
        reinit = True,
        name = name,
        config={
            "steps": 100,
            "learning_rate": 0.2
        }
    )
    
    return run 

In [67]:
ORIG_SHAPE = (512, 512, 3)
DATA_ROOT = 'data/radek'
# Setup
device = torch.device("cuda:0")
torch.cuda.set_device(device)

# Set paths
master_scene = 'data/radek/00008_mesh88_animated.xml'
mesh, cameras, optmization_input = mitsuba_scene_to_torch_3d(master_scene)

In [68]:
raster_settings = RasterizationSettings(image_size=512, blur_radius=0.0, faces_per_pixel=1)
lights = PointLights(device=device, location=[[0.0, 0.0, 3.0]])

renderer = MeshRenderer(
    rasterizer=MeshRasterizer(
        cameras=cameras, 
        raster_settings=raster_settings
    ), 
    shader=HardPhongShader(
        device=device, 
        cameras=cameras, 
        lights=lights
    )
)


Text(0.5, 1.0, 'Pytorch3d')

In [69]:
image = renderer(mesh)   # (1, H, W, 4)
ref_image = cv2.imread('data/radek/out.png')
ref_image = cv2.cvtColor(ref_image, cv2.COLOR_BGR2RGB)

fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(1, 2, 1)
ax.imshow(ref_image)

ax.set_title("Mitsuba (ref)")

ax = fig.add_subplot(1, 2, 2)
ax.imshow(image.squeeze().cpu().numpy())
ax.set_title("Pytorch3d")

In [78]:
ORIG_SHAPE = (512, 512, 3)

body_estimation = Body('/home/ax/data/programs/pytorch-openpose/model/body_pose_model.pth', True)

# set defaults
body_estimation.imageToTest_padded_shape = ORIG_SHAPE
body_estimation.pad = [0, 0, 0, 0]

# Initialize a model using the renderer, mesh and reference image
model = Model(
    texTshirt=optmization_input["texTshirt"], 
    texOther=optmization_input["texOther"], 
    verticies=optmization_input["verticies"], 
    faces=optmization_input["faces"], 
    renderer=renderer,
    body_estimation=body_estimation
).to(device)

# Create an optimizer. Here we are using Adam and we pass in the parameters of the model
optimizer = torch.optim.Adam(model.parameters(), lr=0.05)
filename_output = "./texture_single_tshirt.gif"
run = wandb_init('torch3d')
writer = imageio.get_writer(filename_output, mode='I', duration=0.3)

In [80]:
for i in tqdm(range(100)):
    optimizer.zero_grad()
    loss, cur_image = model()
    loss.backward()
    optimizer.step()
    wandb.log({'loss': loss.item()}, step=i)
    
    # Save outputs to create a GIF. 
    if i % 10 == 0:
        image = cur_image[0, ..., :3].detach().squeeze().cpu().numpy()
        image = np.clip(image, -1, 1)
        image = img_as_ubyte(image)
        writer.append_data(image)
    
writer.close()



  0%|          | 0/100 [00:00<?, ?it/s]wandb: WARNING Adding to old History rows isn't currently supported.  Step 0 < 91; dropping {'loss': 0.04955591633915901}.


  1%|          | 1/100 [00:00<01:00,  1.64it/s]wandb: WARNING Adding to old History rows isn't currently supported.  Step 1 < 91; dropping {'loss': 0.04954209923744202}.


  2%|▏         | 2/100 [00:01<00:57,  1.71it/s]wandb: WARNING Adding to old History rows isn't currently supported.  Step 2 < 91; dropping {'loss': 0.04954180121421814}.


  3%|▎         | 3/100 [00:01<00:55,  1.75it/s]wandb: WARNING Adding to old History rows isn't currently supported.  Step 3 < 91; dropping {'loss': 0.049524787813425064}.


  4%|▍         | 4/100 [00:02<00:53,  1.78it/s]wandb: WARNING Adding to old History rows isn't currently supported.  Step 4 < 91; dropping {'loss': 0.049517594277858734}.


  5%|▌         | 5/100 [00:02<00:52,  1.80it/s]wandb: WARNING Adding to old History rows isn't currently supported.  Step 5 < 91; dropping {'los

In [72]:
(canvas_orig, 
 candidate_orig,
 subset_orig,
 heatmap_avg_orig,
 paf_avg_orig) = get_body_image_from_mesh(mesh, body_estimation, renderer)

(canvas_noise, 
 candidate_noise,
 subset_noise,
 heatmap_avg_noise,
 paf_avg_noise) = get_body_image_from_mesh(model.cur_mesh, body_estimation, renderer)

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


Text(0.5, 1.0, 'Adv')

In [73]:
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(1, 2, 1)
ax.imshow(canvas_orig)
ax.set_title('Original')

ax = fig.add_subplot(1, 2, 2)
ax.imshow(canvas_noise)
ax.set_title('Adv')